In [1]:
import logging
import os
from pathlib import Path
import sqlite3 as sql

from dash import Dash, html, dash_table, dcc, callback, Output, Input
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc


In [2]:
external_stylesheets = [dbc.themes.JOURNAL]
app = Dash(__name__, external_stylesheets=external_stylesheets)

In [3]:
output_folder = Path(os.getenv('GRADDNODI_OUTPUT', 'Output/'))
output_options = [str(dir.parts[-1]) for dir in output_folder.glob('./*') if dir.is_dir()]

In [4]:
@callback(
    Output('folder-path', 'data'),
    Input('folder-name', 'value')
)
def folder_path(name):
    return str(output_folder.joinpath(name))

@callback(
    Output('folder-output-text', 'children'),
    Input('folder-path', 'data') 
)
def folder_path_text(path):
    return f'Using data from {path}'

@callback(
    Output('db-index', 'data'),
    Input('folder-path', 'data')
    )
def get_df_index(path):
    con = sql.connect(Path(path).joinpath('Results').joinpath('Results.db'))
    raw_index = pd.read_sql(
        sql='SELECT "Reference", "Field", "Calibrated", "Technique", "Scaling Method", "Variables" FROM Results;',
        con=con
    )
    con.close()
    return raw_index.to_json(orient='split')

@callback(Output('debug-table', 'figure'), Input('db-index', 'data'))
def debug_table(jsonified_cleaned_data):
    df = pd.read_json(jsonified_cleaned_data, orient='split')
    fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=df.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
    ])
    return fig

@callback(
    Output('reference-options', 'options'),
    Input('db-index', 'data')
)
def ref_opts(data):
    df = pd.read_json(data, orient='split')
    return [
        {'label': i, 'value': i} for i in df['Reference'].unique()
    ]

@callback(
    Output('field-options', 'options'),
    Output('calibrated-device-options', 'options'),
    Output('technique-options', 'options'),
    Output('scaling-options', 'options'),
    Output('var-options', 'options'),
    Input('db-index', 'data'),
    Input('reference-options', 'value'),
    Input('field-options', 'value'),
    Input('calibrated-device-options', 'value'),
    Input('technique-options', 'value'),
    Input('scaling-options', 'value'),
)
def filter_options(data, ref_d, fields, cal_d, tech, sca):
    levels = {
        "Field": fields,
        "Calibrated": cal_d,
        "Technique": tech,
        "Scaling Method": sca,
    }
    db_index = pd.read_json(data, orient='split')
    df = db_index[db_index['Reference'] == ref_d]
    for name, col in levels.items():
        if fields is None:
            cols = list(df[name].unique())
        else:
            cols = col
        df = df[df[name].isin(cols)]
    return (
        [{'label': i, 'value': i} for i in df['Field'].unique()],
        [{'label': i, 'value': i} for i in df['Calibrated'].unique()],
        [{'label': i, 'value': i} for i in df['Technique'].unique()],
        [{'label': i, 'value': i} for i in df['Scaling Method'].unique()],
        [{'label': i, 'value': i} for i in df['Variables'].unique()]
    )
        

In [5]:
item_stores = [
    dcc.Store(id='folder-path'),
    dcc.Store(id='db-index'),
]

top_row = [
    dbc.Row([html.Div('Graddnodi', className='h1', style={'text-align': 'center'})]),
    dbc.Row(
        [
            dbc.Col(
                [
                    dcc.Dropdown(
                        sorted(output_options),
                        sorted(output_options)[0],
                        id='folder-name'
                    ),
                ]
            ),
            dbc.Col(
                [
                    html.Div(id='folder-output-text')
                ]
            )
        ]
    )
    
]


selections = [
    dbc.Row(
        [
            dbc.Col(
                [
                    dcc.RadioItems(id='reference-options')        
                ]
            ),
            
            dbc.Col(
                [
                    dcc.Checklist(id='field-options')        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='calibrated-device-options')        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='technique-options')        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='scaling-options')        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='var-options')        
                ]
            ),
            
        ]
    )
]

app.layout = dbc.Container(
    [
        *item_stores,
        *top_row,
        html.Hr(),
        *selections,
        html.Hr(),
        dbc.Row(
        [
            dcc.Graph(figure={}, id='debug-table')
        ]
    )
    ]
)
"""
    dbc.Row([
        html.Div('Graddnodi', className="text-primary text-center fs-3")
    ]),

    dbc.Row(radio_options, justify="center", align="start"),
    dbc.Row(
        [
            dcc.Graph(figure={}, id='error-table')
        ]
    ),
    dbc.Row(
        [
            dbc.Col([dcc.Graph(figure={}, id='box-plot-rel')]),
            dbc.Col([dcc.Graph(figure={}, id='box-plot-raw')]),
        ]
    ),
    dcc.Store(id='filtered_df')

], fluid=True)
"""

'\n    dbc.Row([\n        html.Div(\'Graddnodi\', className="text-primary text-center fs-3")\n    ]),\n\n    dbc.Row(radio_options, justify="center", align="start"),\n    dbc.Row(\n        [\n            dcc.Graph(figure={}, id=\'error-table\')\n        ]\n    ),\n    dbc.Row(\n        [\n            dbc.Col([dcc.Graph(figure={}, id=\'box-plot-rel\')]),\n            dbc.Col([dcc.Graph(figure={}, id=\'box-plot-raw\')]),\n        ]\n    ),\n    dcc.Store(id=\'filtered_df\')\n\n], fluid=True)\n'

In [6]:
app.run(jupyter_mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/
